In [ ]:
import pandas as pd
import mysql.connector
import os
from dotenv import load_dotenv
import firebase_admin
from firebase_admin import credentials, storage
from datetime import datetime
from google.cloud import bigquery
import numpy as np
from io import StringIO

In [2]:
! pip list

Package                                  Version
---------------------------------------- -----------
aiohttp                                  3.9.5
aiosignal                                1.3.1
alembic                                  1.13.1
annotated-types                          0.6.0
ansicolors                               1.1.8
anyio                                    4.3.0
apache-airflow                           2.9.1
apache-airflow-providers-common-io       1.3.1
apache-airflow-providers-common-sql      1.12.0
apache-airflow-providers-fab             1.0.3
apache-airflow-providers-ftp             3.8.0
apache-airflow-providers-http            4.10.1
apache-airflow-providers-imap            3.5.0
apache-airflow-providers-papermill       3.7.0
apache-airflow-providers-smtp            1.6.1
apache-airflow-providers-sqlite          3.7.1
apispec                                  6.6.0
argcomplete                              3.3.0
asgiref                                  3.8.1
as


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
load_dotenv()

# Extract Data

In [ ]:
def ingestion(table):
    load_dotenv()
    connection = mysql.connector.connect(
        host=os.getenv('HOST'),
        user='root',
        password=os.getenv('PASSWORD'),
        database='capstone5'
    )
    query = f"SELECT * FROM {table}"  
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [ ]:
# List of table names
tables = [
    'applications', 'articles', 'comments', 'donation_manual_comments',
    'donation_manuals', 'fundraising_categories', 'fundraisings',
    'likes_comments', 'organizations', 'testimoni_volunteers',
    'user_bookmark_fundraisings', 'user_bookmark_volunteer_vacancies',
    'volunteers', 'users', 'admins', 'like_donation_comments',
    'user_bookmark_articles', 'user_bookmark_articles'
]

# Ingest data for all tables
data_frame = [ingestion(table) for table in tables]

# Load Raw Data To Firebase

In [ ]:
def initialize_firebase():
    if not firebase_admin._apps:
        credentials_path = os.getenv('FIREBASE_CREDENTIALS_PATH')
        cred = credentials.Certificate(credentials_path)
        firebase_admin.initialize_app(cred)
        print("Firebase has been initialized")
    else:
        print("Firebase is already initialized")
    bucket_name = os.getenv('BUCKET_NAME')
    return storage.bucket(bucket_name)

In [ ]:
def create_folder_in_bucket():
    bucket = initialize_firebase()
    current_date = datetime.now().strftime("%Y%m%d")
    folder_blob = bucket.blob(f"{current_date}/")
    folder_blob.upload_from_string('')
    print(f"Folder '{current_date}' created successfully.")
    return current_date

In [ ]:
def upload_dataframes_to_firebase(data_frames):
    current_date = create_folder_in_bucket()
    bucket = initialize_firebase()
    
    for df, table_name in zip(data_frames, tables):
        # Convert dataframe to CSV string
        csv_str = df.to_csv(index=False)

        # Create the blob reference with folder name
        file_name_with_date = f"{table_name}_{current_date}.csv"
        file_path_in_bucket = f"{current_date}/{file_name_with_date}"
        file_ref = bucket.blob(file_path_in_bucket)
        
        # Upload CSV string to Firebase
        file_ref.upload_from_string(csv_str, content_type='text/csv')
        print(f"Dataframe {table_name} uploaded successfully as {file_name_with_date}!")

In [ ]:
# Upload all dataframes to Firebase
upload_dataframes_to_firebase(data_frame)

# Transform

### Handle Missing Value & Duplicate

In [ ]:
# Cek duplikat dan missing value

def cleaning_data(df):
    
    # cek duplikat
    duplicates = df[df.duplicated(subset=df.columns, keep=False)]
    if not duplicates.empty:
        print(f"Terdapat duplikat pada {df}")
        print(duplicates)
        df = df.drop_duplicates()
    else:
        print(f"Tidak ada data duplikat pada {df}")
        
    # Ubah tipe data
    columns_datetime = ['created_at', 'updated_at', 'deleted_at']
    for col in columns_datetime:
        if df[col].dtype != 'datetime64[ns]':
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
        df[col] = df[col].dt.strftime('%Y-%m-%d')
    

    # cek missing value
    missing_columns = [col for col in df.columns if col not in ['updated_at', 'deleted_at']]
    # df[missing_columns] = df[missing_columns].replace('', None)
    if df[missing_columns].isnull().any().any():
        print(f"Terdapat missing value dalam {df}")
        missing_sum = df[missing_columns].isnull().sum()
        print(missing_sum)
        
        columns_numeric = ['total_likes', 'amount', 'goal_amount', 'current_progress', 'is_verified', 'registered_volunteer']
        for col in missing_columns:
            if df[col].dtype != 'object' and df[col].dtype != 'datetime64[ns]' and col not in columns_numeric:
                df[col] = df[col].fillna(np.nan)
            elif col in columns_numeric :
                df[col] = df[col].fillna(0)
            elif df[col].dtype == 'datetime64[ns]':
                df[col] = df[col].fillna(pd.NaT)
            else:
                df[col] = df[col].fillna('Unknown')
    else :
        print(f"Tidak ada missing value dalam {df}")
        missing_sum = df[missing_columns].isnull().sum()
        print(missing_sum)
    return df

In [ ]:
for df in data_frame :
    cleaning_data(df)

In [ ]:
df_applications = data_frame[0]
df_articles = data_frame[1]
df_comments = data_frame[2]
df_donation_manuals = data_frame[4]
df_fundraising_categories = data_frame[5]
df_fundraisings = data_frame[6]
df_like_comments = data_frame[7]
df_organizations = data_frame[8]
df_testimoni_volunteers = data_frame[9]
df_user_bookmark_fundraisings = data_frame[10]
df_user_bookmark_volunteer_vacancies = data_frame[11]
df_volunteers = data_frame[12]
df_users = data_frame[13]
df_user_bookmark_articles = data_frame[16]

### Tabel Fakta

#### 1. FactDonationTransaction

In [ ]:
# Buat Struktur kolom fact_donation_transaction
columns = ['id', 'donation_id', 'fundraising_id', 'user_id', 'amount', 'goal_amount', 'fundraising_category_id', 'organization_id', 'created_at']
fact_donation = pd.DataFrame(columns=columns)

# mengambil data yang sukses di df_donation 
df_donation_success = df_donation_manuals.loc[df_donation_manuals['status'] == 'sukses']
df_donation_success = df_donation_success.reset_index(drop=True)

# mengisi data pada dari kolom df_donation
fact_donation['id'] = range(1, len(df_donation_success) + 1)
fact_donation['donation_id'] = df_donation_success['id']
fact_donation['fundraising_id'] = df_donation_success['fundraising_id']
fact_donation['user_id'] = df_donation_success['user_id']
fact_donation['amount'] = df_donation_success['amount']
fact_donation['created_at'] = df_donation_success['created_at']

# merge df_fundraising
df_merge_fact_fundraising = pd.merge(fact_donation, df_fundraisings, left_on='fundraising_id', right_on='id', how='left')
fact_donation['goal_amount'] = df_merge_fact_fundraising['goal_amount_y']
fact_donation['fundraising_category_id'] = df_merge_fact_fundraising['fundraising_category_id_y']
fact_donation['organization_id'] = df_merge_fact_fundraising['organization_id_y']

#### 2. FactVolunteerApplications

In [ ]:
# Buat Struktur kolom df_fact_volunteer_applications
columns = ['id', 'application_id', 'vacancy_id', 'user_id', 'organization_id', 'created_at']
fact_applications = pd.DataFrame(columns=columns)

# mengisi data pada dari kolom df_application
fact_applications['id'] = range(1, len(df_applications) + 1)
fact_applications['application_id'] = df_applications['id']
fact_applications['vacancy_id'] = df_applications['vacancy_id']
fact_applications['user_id'] = df_applications['user_id']
fact_applications['created_at'] = df_applications['created_at']

# merge df_fundraising
df_merge_fact_volunteer = pd.merge(fact_applications, df_volunteers, left_on='vacancy_id', right_on='id', how='left')
fact_applications['organization_id'] = df_merge_fact_volunteer['organization_id_y']

#### 3. FactVolunteerTestimoni

In [ ]:
# Buat Struktur kolom df_fact_volunteer_testimoni
columns = ['id', 'user_id', 'vacancy_id', 'testimoni_volunteer_id', 'rating', 'created_at']
fact_volunteer_testimoni = pd.DataFrame(columns=columns)

# mengisi data pada dari kolom df_application
fact_volunteer_testimoni['id'] = range(1, len(df_testimoni_volunteers) + 1)
fact_volunteer_testimoni['user_id'] = df_testimoni_volunteers['user_id']
fact_volunteer_testimoni['vacancy_id'] = df_testimoni_volunteers['vacancy_id']
fact_volunteer_testimoni['testimoni_volunteer_id'] = df_testimoni_volunteers['id']
fact_volunteer_testimoni['rating'] = df_testimoni_volunteers['rating']
fact_volunteer_testimoni['created_at'] = df_testimoni_volunteers['created_at']

#### 4. FactArticlePopular

In [ ]:
# Buat Struktur kolom df_fact_articel_popular
columns = ['id', 'article_id', 'bookmark_id', 'user_id','comment_id', 'like_comment_id', 'created_at']
fact_article_popular = pd.DataFrame(columns=columns)

# mengisi data pada dari kolom df_comment
fact_article_popular['id'] = range(1, len(df_user_bookmark_articles) + 1)
fact_article_popular['article_id'] = df_user_bookmark_articles['article_id']
fact_article_popular['bookmark_id'] = df_user_bookmark_articles['id']
fact_article_popular['user_id'] = df_user_bookmark_articles['user_id']
fact_article_popular['comment_id'] = df_comments['id']
fact_article_popular['like_comment_id'] = df_like_comments['id']
fact_article_popular['created_at'] = df_user_bookmark_articles['created_at']

#### 5. FactBookmarkFundraising

In [ ]:
fact_bookmark_fundraising = df_user_bookmark_fundraisings.drop(['deleted_at', 'updated_at'], axis=1)
fact_bookmark_fundraising = fact_bookmark_fundraising.rename(columns={'id':'bookmark_id'})
fact_bookmark_fundraising['id'] = range(1, len(fact_bookmark_fundraising) + 1)
fact_bookmark_fundraising.insert(0, 'id', fact_bookmark_fundraising.pop('id'))

#### 6. FactBookmarkVolunteer

In [ ]:
fact_bookmark_volunteer_vacancies = df_user_bookmark_volunteer_vacancies.drop(['deleted_at', 'updated_at','volunteer_vacancy_id'], axis=1)
fact_bookmark_volunteer_vacancies = fact_bookmark_volunteer_vacancies.rename(columns={'id':'bookmark_id'})
fact_bookmark_volunteer_vacancies['id'] = range(1, len(fact_bookmark_volunteer_vacancies) + 1)
fact_bookmark_volunteer_vacancies.insert(0, 'id', fact_bookmark_volunteer_vacancies.pop('id'))

### Tabel Dimensi

In [ ]:
dim_fundraisings = df_fundraisings.drop(['fundraising_category_id','organization_id','updated_at'], axis=1)
dim_fundraising_categories = df_fundraising_categories[['id','name','created_at']]
dim_donation_manual = df_donation_manuals.drop(['fundraising_id', 'user_id','updated_at'], axis=1)
dim_organization = df_organizations.drop(['updated_at'], axis= 1)
dim_user = df_users.drop(['updated_at'], axis=1)
dim_volunteer_applictaion = df_applications.drop(['user_id','vacancy_id','updated_at'], axis=1)
dim_volunteer_vacancies = df_volunteers.drop(['organization_id','updated_at'], axis=1)
dim_testimoni_volunteer = df_testimoni_volunteers.drop(['user_id','vacancy_id','updated_at'], axis=1)
dim_article = df_articles.drop(['updated_at'], axis = 1)
dim_bookmark_fundraising = df_user_bookmark_fundraisings.drop(['fundraising_id','user_id','updated_at'], axis=1)
dim_bookmark_volunter_vacancies = df_user_bookmark_volunteer_vacancies.drop(['volunteer_vacancies_id','user_id','updated_at'], axis=1)
dim_bookmark_article = df_user_bookmark_articles.drop(['deleted_at','updated_at'], axis=1)

dim_comments = df_comments.drop(['user_id','article_id','updated_at'], axis=1)
dim_like_comments = df_like_comments.drop(['user_id','comment_id', 'updated_at'], axis=1)

In [ ]:
df_fact = [
    ('fact_applications', fact_applications),
    ('fact_article_popular', fact_article_popular),
    ('fact_bookmark_fundraising', fact_bookmark_fundraising),
    ('fact_bookmark_volunteer_vacancies', fact_bookmark_volunteer_vacancies),
    ('fact_donation', fact_donation),
    ('fact_volunteer_testimoni', fact_volunteer_testimoni)
]

df_dim = [
    ('dim_article', dim_article),
    ('dim_bookmark_fundraising', dim_bookmark_fundraising),
    ('dim_bookmark_volunter_vacancies', dim_bookmark_volunter_vacancies),
    ('dim_bookmark_article', dim_bookmark_article),
    ('dim_donation_manual', dim_donation_manual),
    ('dim_fundraising_categories', dim_fundraising_categories),
    ('dim_fundraisings', dim_fundraisings),
    ('dim_organization', dim_organization),
    ('dim_testimoni_volunteer', dim_testimoni_volunteer),
    ('dim_user', dim_user),
    ('dim_volunteer_application', dim_volunteer_applictaion),
    ('dim_volunteer_vacancies', dim_volunteer_vacancies),
    ('dim_comments', dim_comments),
    ('dim_like_comments', dim_like_comments)
]

# Load

### Load To Bigquery

In [ ]:
credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if credentials_path:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
else:
    raise Exception("GOOGLE_APPLICATION_CREDENTIALS is not set in the .env file")

client = bigquery.Client()

dataset_id_fact = os.getenv("dataset_id_fact")
dataset_id_dim = os.getenv("dataset_id_dim")

In [ ]:
def load_df_to_bigquery(dataset_id, table_name, df):
    # Create a BigQuery client
    client = bigquery.Client()

    # Convert DataFrame to CSV
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)

    table_id = f"{dataset_id}.{table_name}"
    
    partition_by = bigquery.TimePartitioning(field="created_at")

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
        time_partitioning=partition_by
    )

    # Load CSV data from StringIO buffer
    job = client.load_table_from_file(csv_buffer, table_id, job_config=job_config)
    
    # Wait for the load job to complete
    job.result()

    # Get table information
    table = client.get_table(table_id)
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

In [ ]:
# Load fact tables
for table_name, df in df_fact:
    load_df_to_bigquery(dataset_id_fact, table_name, df)

# Load dimension tables
for table_name, df in df_dim:
    load_df_to_bigquery(dataset_id_dim, table_name, df)